In [2]:
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pathlib
from pathlib import Path
from sklearn.model_selection import train_test_split
import random
import requests
import scipy as sp
import tensorflow as tf
import tensorflow.keras as keras
import zipfile

In [3]:
os.environ['TF_CPP_MIN_LEVEL'] = '2'

In [6]:
trainBlotchDirectory = Path('C:/Users/Anton/MakeAIWork2/projects/apple_disease_classification/data/Train/Blotch_Apple')
trainNormalDirectory = Path('C:/Users/Anton/MakeAIWork2/projects/apple_disease_classification/data/Train/Normal_Apple')
trainRotDirectory = Path('C:/Users/Anton/MakeAIWork2/projects/apple_disease_classification/data/Train/Rot_Apple')
trainScabDirectory = Path('C:/Users/Anton/MakeAIWork2/projects/apple_disease_classification/data/Train/Scab_Apple')

edgeFiles = list()
imageList = list()
for filename in os.listdir(trainBlotchDirectory):
    imgFile = os.path.join(trainBlotchDirectory, filename)
    edgeFiles.append(imgFile)

for filename in os.listdir(trainNormalDirectory):
    imgFile = os.path.join(trainNormalDirectory, filename)
    edgeFiles.append(imgFile)

for filename in os.listdir(trainRotDirectory):
    imgFile = os.path.join(trainRotDirectory, filename)
    edgeFiles.append(imgFile)

for filename in os.listdir(trainScabDirectory):
    imgFile = os.path.join(trainScabDirectory, filename)
    edgeFiles.append(imgFile)

print(edgeFiles[0])

C:\Users\Anton\MakeAIWork2\projects\apple_disease_classification\data\Train\Blotch_Apple\10.jpg


In [9]:
edgeFiles = [files.replace("\\","/") for files in edgeFiles]
edgeFiles = [img for img in edgeFiles if ".jpg" in img]

print(edgeFiles[0])

C:/Users/Anton/MakeAIWork2/projects/apple_disease_classification/data/Train/Blotch_Apple/10.jpg


In [17]:
imageObjects = np.zeros([len(edgeFiles), 64, 64, 3])

i = 0

for pic in edgeFiles:
  imageObjects[i] = np.asarray(Image.open(pic)).astype('uint8')/255
  i += 1
    


ValueError: could not broadcast input array from shape (575,767,3) into shape (64,64,3)

In [ ]:
imageLabels = np.empty(len(edgeFiles), dtype = 'S20')

i = 0

for label in edgeFiles:
    edgeFiles[i] = label.split('/')[1]
    i += 1
    

labelNames, labelNumbers = np.unique(edgeFiles, return_inverse=True)
labelDict = dict(zip(np.unique(labelNumbers), labelNames))
print(labelNames)
print(labelNumbers)

np.array(np.unique(labelNumbers, return_counts=True)).T

In [ ]:
def inspectData():
    
    plt.figure(figsize=(10,10))
    
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(imageObjects[i])
        plt.xlabel(labelNames[labelNumbers[i]]) 

    plt.show()   

In [ ]:
inspectData() 

In [ ]:
num_classes = len(labelNames)

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes))

model.summary()

In [ ]:
lossFunction = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gdAlgorithm = keras.optimizers.Adam(learning_rate=0.001)
nrOfEpochs = 5

In [ ]:
model.compile(optimizer=gdAlgorithm, loss=lossFunction, metrics="accuracy")
history = model.fit(trainSet, trainLabels, epochs=nrOfEpochs, batch_size=64, verbose=2)

In [ ]:
model.compile(optimizer=gdAlgorithm, loss=lossFunction, metrics="accuracy")
history_test = model.fit(testSet, testLabels, epochs=nrOfEpochs, batch_size=64, verbose=2)

In [ ]:
trainSet, testSet, trainLabels, testLabels = train_test_split(imageObjects, labelNumbers, stratify = labelNumbers, train_size = 0.75, random_state=42)
print(trainSet.shape)
nrOfImages = len(trainSet)
print(nrOfImages)

In [ ]:
#hier kies je welke index je iets mee wilt
i = 10

#dit is het plaatje van index i
img = trainSet[i]

# print de label naam van index i
print(labelNames[trainLabels[i]])

#laat plaatje zien
plt.imshow(img);

#iets met predict doen

In [ ]:
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


In [ ]:
#get predictions
pred = model.predict(testSet, batch_size=16)
pred = np.argmax(pred, axis=-1)

#original labels
labels = np.argmax(testLabels,axis=-1)
from sklearn.metrics import classification_report
print(classification_report(labels, pred))